In [ ]:
from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
import extGoogleDrive as extGoogleDrive
# from extGoogleDrive import extGoogleDrive as GoogleDrive
from pprint import pprint
import pyAesCrypt
import io
import os
import hashlib
import numpy as np

from helpers import mkdir, driveList2localDict, ignore, strip_aes, file_size, size_to_upload
from helpers import obfuscate, deobfuscate

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = extGoogleDrive.extGoogleDrive(gauth)

## Drive class handler

In [ ]:
class DriveHandler:
    
    def __init__(self, local_root, password = "foopassword", bufferSize=64*1024, cipher=True):
        self.root_id = "1YJkqN-qtRHXnL4GvT3icGyl05_dcYTE6"
        self.local_root = local_root
        self.cache = None
        self.bufferSize = bufferSize
        self.password = hashlib.sha256(password.encode("utf-8")).hexdigest()
#         self.drive = drive
        self.cipher = cipher

# d = DriveHandler("./mypydrive_1")

## List local files

In [ ]:
def get_local_folder(self, verbose=False):
    folder_list= {self.local_root: ""}
    def localListFolder(path_structure, verbose=verbose):
        local_folders = [d for d in os.listdir(path_structure) if os.path.isdir(os.path.join(path_structure, d))]
        for f in local_folders:
            if ignore(f):
                if verbose: print(f"{path_structure}/{f}: {f['id']}")
                folder_list[f"{path_structure}/{f}"] = ""
                localListFolder(f"{path_structure}/{f}", verbose=verbose)
            
    localListFolder(self.local_root, verbose=verbose)
    return folder_list

# get_local_folder(d)

In [ ]:
def get_local_files(self):
    folders = get_local_folder(self)
    lfile_list = {}
    for fpath, _ in folders.items():
        lfile_list[fpath] = {d: "" for d in os.listdir(fpath) if ignore(d) and os.path.isfile(os.path.join(fpath, d))}
    return lfile_list
        
# get_local_files(d)

## List remote folders

In [ ]:
def get_remote_folders(self, decipher=True, verbose=True):
    folder_list= {"remote_root": self.root_id}
    if verbose: print(f"remote_root: '{self.root_id}'")
    
    def driveListFolder( parent, path_structure, verbose=verbose, decipher=decipher):
        remote_folders = drive.ListFile({'q': f"'{parent}' in parents and trashed=false and mimeType='application/vnd.google-apps.folder'"}).GetList()
        for f in remote_folders:
            title = f['title']
            if decipher: title = deobfuscate(self.password, title)
            if verbose: print(f"{path_structure}/{title}: {f['id']}")
            folder_list[f"{path_structure}/{title}"] = f['id']
            # folder_list.append({f"{path_structure}/{title}": f['id']})
            driveListFolder(f['id'], f"{path_structure}/{title}", verbose, decipher)
        
    driveListFolder(self.root_id, "remote_root", decipher=decipher, verbose=verbose)
    return folder_list
            

# r_folders = get_remote_folders(d, verbose=True)
# r_folders

## List remote Files

In [ ]:
def get_remote_files(self, remote_folders, verbose=False):
    r_files = {}
    decipher = True
    for folder_name, folder_id in remote_folders.items():
        files = {}
        remote_files = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false and mimeType!='application/vnd.google-apps.folder'"}).GetList()
        for f in remote_files:
            title = f['title']
            if decipher: title = deobfuscate(self.password, strip_aes(title))
#             files.append({f"{title}": f['id']})
            files[f"{title}"] = f['id']
            if verbose: print(f'REMOTE LS: {folder_name} {title}')
        r_files[folder_name] = files
    return r_files
        
# r_files = get_remote_files(d, r_folders)
# r_files

# To upload

## Upload folders

In [ ]:
def folders_to_create(self, r_folders, verbose=False):
    L_folders = get_local_folder(self)
    to_create = []
    for folder in L_folders:
        rf = folder.replace(self.local_root, "remote_root")
        if rf not in r_folders.keys():
            if verbose: print(f"TODO REMOTE CREATE: {rf}" )
            to_create.append(rf)
    return to_create


# folders_to_create(d, r_folders)

In [ ]:
def create_remote_folder(self, r_folders, cipher=True, verbose=True):
    to_create = folders_to_create(self, r_folders)
    for f in to_create:
        dirs = f.split("/")
        n = len(dirs)
        for i in range(n):
            remote_path = '/'.join(dirs[:i+1])
            if remote_path in r_folders.keys():
                continue
            else:
                parent_id = r_folders['/'.join(dirs[:i])]
                if cipher: child_name = obfuscate(self.password, dirs[i])
                child_folder = drive.CreateFile({'title': child_name, 
                                                  'parents':[{'id':parent_id}], 
                                                  'mimeType' : 'application/vnd.google-apps.folder'})
                child_folder.Upload()
                r_folders[remote_path] = child_folder['id']
                if verbose: print(f"REMOTE CREATE FOLDER: {remote_path} | {child_name} | p:{parent_id} | c:{child_folder['id']}" )
                # print(remote_path, child_name, parent_id, child_folder['id'])
            
# create_remote_folder(d, r_folders)
# r_folders

## Upload Files

In [ ]:
def encript_and_upload(self, fpath, fname, parent_id):
    fIn = open(f"{fpath}/{fname}", "rb")
    fCiph = io.BytesIO()
    pyAesCrypt.encryptStream(fIn, fCiph, self.password, self.bufferSize)

    if self.cipher: fname = obfuscate(self.password, fname)
    file = drive.CreateFile({'title': f"{fname}.aes", 
                              'parents':[{'id': parent_id}]
                                 })
    file.SetContentBinary(fCiph)
    file.Upload() # Actual Upload

    fIn.close()
    return file['id']

In [ ]:
def files_to_upload(self, r_folders, r_files, verbose=False):
    to_upload = []
    for folder, files in get_local_files(self).items():
        lf = folder
        rf = folder.replace(self.local_root, "remote_root")
        items = r_files.get(rf , {})
        
        for f in files:
            if f not in items.keys():
                to_upload.append( (lf, f, r_folders[rf]) )
                if verbose: print(f"TO UPLOAD: {lf}/{f} | Remote Folder ID: {r_folders[rf]}")
                
    return to_upload

# f2upload = files_to_upload(d, r_files)
# f2upload

In [ ]:
def upload_files(self, f2upload, verbose=False):
    total = size_to_upload(f2upload, in_bytes=True)
    print(f'Total to upload: {file_size(total)}')
    for fpath, fname, parent_id in f2upload:
        file_id = encript_and_upload(self, fpath, fname, parent_id)
        total -= file_size(f'{fpath}/{fname}', in_bytes=True)
        fsize = file_size(f'{fpath}/{fname}')
        print(f"{file_size(total):8} | {file_id} | ({fsize}) {fname}")
    
# upload_files(d, f2upload)

## Files to remove

In [ ]:
def folders_to_delete(self, r_folders, verbose=False):
    L_folders = get_local_folder(self)
    to_delete = []
    for folder, folder_id in r_folders.items():
        rf = folder.replace("remote_root", self.local_root)
        if rf not in L_folders.keys():
            to_delete.append((folder, folder_id))
            if verbose: print(f"TODO REMOTE DELETE: {folder} | {folder_id}")
    return to_delete

# to_delete = folders_to_delete(d, r_folders)
# to_delete

In [ ]:
def delete_remote_folders(self, to_delete, verbose=True):
    for folder, folder_id in to_delete:
        file = drive.CreateFile({'id': folder_id})
        file.Trash()
        if verbose: print(f"REMOTE TRASH: {folder} | {folder_id}")

# delete_remote_folders(d, to_delete)

In [ ]:
def files_to_delete(self, r_folders, r_files, verbose=False):
    L_files = get_local_files(self)
    to_delete = []
    for folder, folder_id in r_folders.items():
        rr_files = r_files[folder]
        LL_files = L_files[folder.replace("remote_root", self.local_root)].keys()
        for r_file, r_id in rr_files.items():
            if r_file not in LL_files:
                to_delete.append( (folder, r_file, r_id))
                if verbose: print(f"TODO REMOTE DELETE: {r_id} | {folder} | {r_file}")
    return to_delete   

# fto_delete = files_to_delete(d, r_folders, r_files)
# fto_delete

In [ ]:
def delete_remote_files(self, to_delete, verbose=False):
    for folder, fname, f_id in to_delete:
        file = drive.CreateFile({'id': f_id})
        file.Trash()
        if verbose: print(f"REMOTE TRASH: {f_id} | {folder}/{fname}")
        
# delete_remote_files(d, fto_delete, verbose=True)

# Mirror Upload

In [ ]:
def upload_mirror(self, verbose=True, interactive=False):
    if verbose: print("# Getting Local Files...")
    local_files = get_local_files(self)
    
    if verbose: print("# Getting Remote Folders...")
    remote_folders = get_remote_folders(self, verbose=verbose)
    if interactive: input("Enter to continue")
    
    if verbose: print("# Creating Remote Folders...")
    create_remote_folder(self, remote_folders, verbose=verbose)
    if interactive: input("Enter to continue")
    
    if verbose: print("# Getting Remote Files...")
    remote_files = get_remote_files(self, remote_folders, verbose=verbose)
    if interactive: input("Enter to continue")
        
    # files to upload
    if verbose: print("# Generating to upload file list...")
    files_2_upload = files_to_upload(self, remote_folders, remote_files, verbose=verbose)
    if interactive: input("Enter to continue")
        
    if verbose: print("# Uploading files...")
    upload_files(self, files_2_upload, verbose=verbose)
    if interactive: input("Enter to continue")
        
    # delete remote folders
    if verbose: print("# Generating to delete remote folder list...")
    f2delete = folders_to_delete(self, remote_folders, verbose=verbose)
    if interactive: input("Enter to continue")
        
    if verbose: print("# Deleting remote folders...")
    delete_remote_folders(self, f2delete, verbose=verbose)
    if interactive: input("Enter to continue")
        
    # delete remote files
    if verbose: print("# Refreashing remote file/folder list...")
    remote_folders = get_remote_folders(self, verbose=verbose)
    remote_files = get_remote_files(self, remote_folders, verbose=verbose)
    if interactive: input("Enter to continue")
        
    if verbose: print("# Generating to delete remote file list...")
    f2_delete = files_to_delete(self, remote_folders, remote_files, verbose=verbose)
    if interactive: input("Enter to continue")
        
    if verbose: print("# Deleting remote files...")
    delete_remote_files(self, f2_delete, verbose=verbose)
    

In [ ]:
d = DriveHandler("./test")
d = DriveHandler("./mypydrive_1")
upload_mirror(d, interactive=True)